# Семинар 6. Линейная алгебра в NumPy

Очень часто различные научные вычисления и алгоритмы связаны с операциями линейной алгебры: перемножение матриц и векторов, вычисление обратных матриц, вычисление собственных значений матрицы, осуществление различных разложений матриц и т.д. Все эти операции играют важную роль, например, в методах оптимизации или глубинном обучении. Именно поэтому основные алгоритмы линейной алгебры были реализованы в виде специального подмодуля библиотеки NumPy - `linalg` (сокращение от *linear algebra*). В сегодняшнем семинаре мы посмотрим на основной функционал этого модуля.

**Необходимые импорты:**

In [1]:
import numpy as np

np.random.seed(42)

## Слияние и деление массивов

Но прежде чем перейти к изучению подмодуля `linalg`, рассмотрим пару полезных функций слияния и разделения матриц. Функции слияния нескольких матриц в одну могут оказаться полезными по ряду причин. Это может быть, как оптимизация вычислений, ведь некоторые функции способны обрабатывать батчи матриц, следовательно, целесообразно объединить матрицы в один батч и передать в такую функцию, чтобы избежать использования циклов. Также это может потребоваться, как необходимый шаг выполнения некоторого алгоритма. Например, для получения расширенной матрицы, мы можем захотеть объединить матрицу коэффициентов с вектором свободных частей.

На прошлом занятии вы познакомились с функцией `np.append`, которая позволяет дописывать значения в конце существующего массива. Похожая на неё функция `np.concatenate` позволяет осуществить те же самые действия. По большому счету, существенной разницы между этими функциями нет, за тем исключением, что в `np.concatenate` вы можете передать аргумент `out`, со значением которого мы знакомились на одном из первых семинаров. Важно отметить тот факт, что объединяемые массивы должны иметь одну и ту же размерность (одно и тоже значение атрибута `ndim`).

In [2]:
array_1d = np.random.randint(-10, 10, size=2)
array_2d = np.random.randint(-10, 10, size=(2, 2))

print(
    f"array_1d:\n{array_1d}",
    f"array_2d:\n{array_2d}",
    f"concatenate-no-axis:\n"
    f"{np.concatenate((array_1d, array_2d), axis=None)}",
    f"concatenate-with-axis:\n"
    f"{np.concatenate((array_2d, array_1d[np.newaxis, ...]), axis=0)}",
    f"append-no-axis:\n"
    f"{np.append(array_1d, array_2d, axis=None)}",
    f"append-with-axis:\n"
    f"{np.append(array_1d[..., np.newaxis], array_2d, axis=1)}",
    sep="\n\n",
)

array_1d:
[-4  9]

array_2d:
[[ 4  0]
 [-3 -4]]

concatenate-no-axis:
[-4  9  4  0 -3 -4]

concatenate-with-axis:
[[ 4  0]
 [-3 -4]
 [-4  9]]

append-no-axis:
[-4  9  4  0 -3 -4]

append-with-axis:
[[-4  4  0]
 [ 9 -3 -4]]


Как вы видите, с помощью добавления новых размерностей и указания аргумента axis мы можем регулировать, как именно значения массивов будут объединяться, и какой формы будет результирующий массив. Однако при работе с матрицами нам не нужен такой уровень гибкости, поскольку мы будем хотеть приписывать или столбцы (`axis=1`), или строки (`axis=0`). Специально для этой цели в NumPy есть пара функций `np.hstack` (сокращение от *horizontal stacking*) и `np.vstack` (аналогично, *vertical stacking*).

In [3]:
array_1d = np.random.randint(-10, 10, size=2)
array_2d = np.random.randint(-10, 10, size=(2, 2))

print(
    f"array_1d:\n{array_1d}",
    f"array_2d:\n{array_2d}",
    "hstack 1d to 2d:\n"
    f"{np.hstack((array_2d, array_1d[..., np.newaxis]))}",
    "hstack 2d to 1d:\n"
    f"{np.hstack((array_1d[..., np.newaxis], array_2d))}",
    "vstack 1d to 2d:\n"
    f"{np.vstack((array_2d, array_1d))}",
    "vstack 2d to 1d:\n"
    f"{np.vstack((array_1d, array_2d))}",
    sep="\n\n",
)

array_1d:
[8 0]

array_2d:
[[ 0 -7]
 [-3 -8]]

hstack 1d to 2d:
[[ 0 -7  8]
 [-3 -8  0]]

hstack 2d to 1d:
[[ 8  0 -7]
 [ 0 -3 -8]]

vstack 1d to 2d:
[[ 0 -7]
 [-3 -8]
 [ 8  0]]

vstack 2d to 1d:
[[ 8  0]
 [ 0 -7]
 [-3 -8]]


Очевидно, если есть возможность слияние массивов в один, также существует возможность разделения одно массива на несколько массивов. Начнем с первого антипода - антипод функции `np.concatenate` - `np.split`. С помощью данной функции вы можете разбить переданный массив на несколько подмассивов по заданному правилу вдоль заданного направления.

In [4]:
def convert_npsplit_to_string(
    parts: list[np.ndarray],
) -> str:
    return ",\n".join(
        [str(part) for part in parts]
    )

In [5]:
array = np.random.randint(-10, 10, size=(6, 4))

print(
    f"array:\n{array}",
    "split-use-int-rule:\n"
    f"{convert_npsplit_to_string(np.split(array, 3))}",
    "split-use-slice-rule:\n"
    f"{convert_npsplit_to_string(np.split(array, [1, 4]))}",
    "split-use-axis:\n"
    f"{convert_npsplit_to_string(np.split(array, 2, axis=1))}",
    sep="\n\n",
)

array:
[[ -9   1  -5  -9]
 [-10   1   1   6]
 [ -1   5   4   4]
 [  8   1   9  -8]
 [ -6   8  -4  -2]
 [ -4   7  -7   3]]

split-use-int-rule:
[[ -9   1  -5  -9]
 [-10   1   1   6]],
[[-1  5  4  4]
 [ 8  1  9 -8]],
[[-6  8 -4 -2]
 [-4  7 -7  3]]

split-use-slice-rule:
[[-9  1 -5 -9]],
[[-10   1   1   6]
 [ -1   5   4   4]
 [  8   1   9  -8]],
[[-6  8 -4 -2]
 [-4  7 -7  3]]

split-use-axis:
[[ -9   1]
 [-10   1]
 [ -1   5]
 [  8   1]
 [ -6   8]
 [ -4   7]],
[[-5 -9]
 [ 1  6]
 [ 4  4]
 [ 9 -8]
 [-4 -2]
 [-7  3]]


Также аналоги-разделители определены и для функций `np.hstack` и `np.vstack` - `np.hsplit` и `np.vsplit`. Обе эти функции аналогичны функции `np.split` с фиксированным значением аргумента axis: axis=0 для vsplit, axis=1 для hsplit.

In [6]:
array = np.random.randint(-10, 10, size=(6, 4))

print(
    f"array:\n{array}",
    "vsplit:\n"
    f"{convert_npsplit_to_string(np.vsplit(array, 3))}",
    "hsplit:\n"
    f"{convert_npsplit_to_string(np.hsplit(array, [1, 3]))}",
    sep="\n\n",
)

array:
[[ 7 -2 -9  9]
 [ 4 -4  1 -3]
 [ 4 -8  3  6]
 [-7  7 -3 -7]
 [-9 -5 -1 -7]
 [ 7  1 -9 -1]]

vsplit:
[[ 7 -2 -9  9]
 [ 4 -4  1 -3]],
[[ 4 -8  3  6]
 [-7  7 -3 -7]],
[[-9 -5 -1 -7]
 [ 7  1 -9 -1]]

hsplit:
[[ 7]
 [ 4]
 [ 4]
 [-7]
 [-9]
 [ 7]],
[[-2 -9]
 [-4  1]
 [-8  3]
 [ 7 -3]
 [-5 -1]
 [ 1 -9]],
[[ 9]
 [-3]
 [ 6]
 [-7]
 [-7]
 [-1]]


## Операции с матрицами

Начнем изучение возможностей NumPy по работе с операциями линейной алгебры с функционала, доступного без подключения подмодуля `linalg`. А именно - рассмотрим все возможные произведения.

### Оператор @ и скалярные произведения

Начиная с версии 3.5 в Python был добавлен новый оператор - @, который в контексте массивов NumPy имеет смысл обычного произведения матриц:

In [7]:
vector1 = np.random.randint(-10, 10, size=3)
vector2 = np.random.randint(-10, 10, size=3)

matrix1 = np.random.randint(-10, 10, size=(4, 3))
matrix2 = np.random.randint(-10, 10, size=(3, 4))

print(
    f"vector1:\n{vector1}",
    f"vector2:\n{vector2}",
    f"matrix1:\n{matrix1}",
    f"matrix2:\n{matrix2}",
    sep="\n\n",
)

vector1:
[-7  3  5]

vector2:
[ 4 -3  3]

matrix1:
[[ -3   5   2]
 [  7   4   2]
 [ -2   4   2]
 [-10  -4  -2]]

matrix2:
[[-10   1  -3   0]
 [  8   6  -3  -8]
 [ -8 -10  -6  -1]]


In [8]:
print(
    f"vector @ vector:\n{vector1 @ vector2}",
    f"vector1 @ matrix2:\n{vector1 @ matrix2}",
    f"matrix1 @ vector2:\n{matrix1 @ vector2}",
    f"matrix @ matrix:\n{matrix1 @ matrix2}",
    sep="\n\n",
)

vector @ vector:
-22

vector1 @ matrix2:
[ 54 -39 -18 -29]

matrix1 @ vector2:
[-21  22 -14 -34]

matrix @ matrix:
[[ 54   7 -18 -42]
 [-54  11 -45 -34]
 [ 36   2 -18 -34]
 [ 84 -14  54  34]]


Из данных примеров видно, что одномерный массив задает как вектор-строки, так и вектор-столбцы, а оператора @ относится к одномерным массивам, как к произведениям вектора-строки на вектор-столбец, т.е. как к обычному скалярному произведению векторов. Но как быть, если мы хотим получить матрицу при выполнении операции @? Т.е., как быть, если нам необходимо осуществить произведение вектор-столбца на вектор-строку? Для решения данной проблемы существует два решения: манипуляция с размерностями и функция `np.outer`. Ниже рассмотрены оба подхода.

In [9]:
vector1 = np.random.randint(-10, 10, size=3)
vector2 = np.random.randint(-10, 10, size=3)

print(
    f"vector1:\n{vector1}",
    f"vector2:\n{vector2}",
    "shape manipulation:\n"
    f"{vector1[..., np.newaxis] @ vector2[np.newaxis, ...]}",
    f"np.outer:\n{np.outer(vector1, vector2)}",
    sep="\n\n",
)

vector1:
[-4 -2 -4]

vector2:
[-2 -3  1]

shape manipulation:
[[ 8 12 -4]
 [ 4  6 -2]
 [ 8 12 -4]]

np.outer:
[[ 8 12 -4]
 [ 4  6 -2]
 [ 8 12 -4]]


Как и в случае с векторизованными операциями, оператор @ имеет функциональную форму `np.matmul`, с помощью которой можно указать буфер записи результата.

### Векторное произведение

Векторное произведение не имеет операторной записи, но имеет функциональную реализациюв в виде функции `np.cross`. Функция принимает на вход массивы различных размерностей и имеет возможности указывать измерения массивов, вдоль которых расположены векторы для осуществления операций.

In [10]:
vector = np.random.randint(-10, 10, size=3)
pack_of_vectors = np.random.randint(-10, 10, size=(3, 3))

print(
    f"vector:\n{vector}",
    f"pack of vectors:\n{pack_of_vectors}",
    f"cross product:\n{np.cross(vector, pack_of_vectors, axis=-1)}",
    sep="\n\n",
)

vector:
[ -9 -10   5]

pack of vectors:
[[ -6  -8   1]
 [ -3  -8 -10]
 [ -8  -6   4]]

cross product:
[[  30  -21   12]
 [ 140 -105   42]
 [ -10   -4  -26]]


### Задание 1. Смешанное произведение

Итак, в NumPy есть скалярное и векторное произведение. Но, к сожалению, нет смешанного произведения векторов. Чтобы исправить это упущение, реализуем функцию для вычисления смешанного произведения, заложив туда возможность вычислять смешанное произведение для целого батча векторов. Итак, необходимо реализовать функцию, на вход которой подается или три вектора, или три матрицы. В случае векторов результат выполнения функции - число - смешанное произведение векторов. В случае матриц результат - n чисел - смешанные произведения. n - число векторов в матрицах.

В случае если матрицы имеют разные размеры, необходимо возбудить исключение `ShapeMismatchError`. В случае, если хотя бы одна из переданных матриц имеет размерность, большую двух, необходимо возбудить исключение `ShapeMismatchError`.

Замечание, считаем, что вектора в матрице храняться вдоль измерения axis=1.

**Напоминание**

Смешнным прозведением векторов $(\vec{a}, \vec{b}, \vec{c})$ называют скалярное прозведение векторов $(\vec{a}, \vec{h})$, где вектор $\vec{h} = \vec{b} \times \vec{c}$ - векторное произведение векторов $\vec{b}, \vec{c}$.

**Необходимые импорты**:

In [11]:
from typing import Union
from numbers import Real

**Программная реализация**:

In [12]:
class ShapeMismatchError(Exception):
    pass

In [13]:
def get_mixed_product(
    vec1: np.ndarray,
    vec2: np.ndarray,
    vec3: np.ndarray,
) -> Union[np.ndarray, Real]:
    # ваш код
    
    return 0

**Тесты**:

In [14]:
vec1 = np.arange(3)
vec2 = np.array([1, 0, 0])
vec3 = np.array([0, 1, 0])

result = get_mixed_product(vec1, vec2, vec3)
assert result == 2

result = get_mixed_product(
    np.repeat(vec1[np.newaxis, ...], 3, axis=0),
    np.repeat(vec2[np.newaxis, ...], 3, axis=0),
    np.repeat(vec3[np.newaxis, ...], 3, axis=0),
)
assert np.all(result == np.full_like(vec1, fill_value=2))

try:
    get_mixed_product(
        np.repeat(vec1[np.newaxis, ...], 3, axis=0),
        vec2,
        vec3,
    )

except ShapeMismatchError:
    pass

else:
    assert False, "exception expected"

try:
    get_mixed_product(
        vec1[np.newaxis, np.newaxis, ...],
        vec2,
        vec3,
    )

except ShapeMismatchError:
    pass

else:
    assert False, "exception expected"

### Вычисление основных чисел матриц

**Норма**

Во многих задач полезным бывает вычислить норму вектора, для последующей нормировки этого вектора. Для этой цели стоит использовать функцию `np.linalg.norm`, которая обладает очень широким функционалом и позволяет вычислять как нормы векторов, так и нормы матриц. Причем, с помощью аргумента `ord` вы можете выбрать, какую именно норму следует вычислить. По умолчанию будет вычислена знакомая вам со школы l2-норма для векторов. В общем же случае эта норма носит название Евклидовой нормы или нормы Фробениуса. Также у этой функции есть аргумент `axis`, благодаря которому вы получаете возможноть вычисления нормы вдоль указанных измерений массива. 

In [15]:
matrix = np.random.randint(-10, 10, size=(3, 3))

print(
    f"matrix:\n{matrix}",
    f"matrix norm:\n{np.linalg.norm(matrix)}",
    f"vectors' norms\n{np.linalg.norm(matrix, axis=-1)}",
    "custom vectors' norm:\n"
    f"{np.sqrt(np.sum(matrix ** 2, axis=-1))}",
    sep="\n\n",
)

matrix:
[[  3  -8 -10]
 [ -6   3  -4]
 [ -2   4   4]]

matrix norm:
16.431676725154983

vectors' norms
[13.15294644  7.81024968  6.        ]

custom vectors' norm:
[13.15294644  7.81024968  6.        ]


**Ранг**

Для выполнения ряда операций требуется определение ранга матрицы. Например, определение ранга матрицы может быть необходимым перед обращением матрицы во время решения матричных уравнений. В NumPy определение ранга матрицы осуществляется с помощью функции `np.linalg.matrix_rank`, в основе которой лежит алгоритм сингулярного разложения матриц.

In [16]:
matrix_full_rank = np.diag(np.random.randint(1, 5, size=4))
matrix_not_full_rank = matrix_full_rank.copy()
matrix_not_full_rank[1, 1] = 0

print(
    f"full-rank matrix:\n{matrix_full_rank}",
    f"not full-rank matrix:\n{matrix_not_full_rank}",
    "full-rank matrix rank:\n"
    f"{np.linalg.matrix_rank(matrix_full_rank)}",
    "not full-rank matrix rank:\n"
    f"{np.linalg.matrix_rank(matrix_not_full_rank)}",
    sep="\n\n",
)

full-rank matrix:
[[2 0 0 0]
 [0 2 0 0]
 [0 0 4 0]
 [0 0 0 1]]

not full-rank matrix:
[[2 0 0 0]
 [0 0 0 0]
 [0 0 4 0]
 [0 0 0 1]]

full-rank matrix rank:
4

not full-rank matrix rank:
3


**Детерминант**

Детерминант матрицы является ее очень важной численной характеристикой. Вычисление детерминанта в NumPy возможно с помощью функции `np.linalg.det`. Однако, поскольку вычисление детерминанта связано с вычислениями произведений, в стех случаях, когда матрица является очень большой, а её элементы - это очень большие числа, в функции `np.linalg.det` может произайти переполнение, и посчитанное значение нельзя будет использовать в дальнейших вычислениях. С этой целью в NumPy реализована специальная функция `np.linalg.slogdet`, которая возвращает знак детерминанта и натуральный логарифм его модуля.

In [17]:
matrix = np.diag(np.random.randint(1, 5, size=4))

print(
    f"matrix:\n{matrix}",
    f"det:\n{np.linalg.det(matrix)}",
    f"slogdet:\n{np.linalg.slogdet(matrix)}",
    sep="\n\n",
)

matrix:
[[3 0 0 0]
 [0 3 0 0]
 [0 0 4 0]
 [0 0 0 3]]

det:
108.00000000000003

slogdet:
SlogdetResult(sign=1.0, logabsdet=4.68213122712422)


**Собственные числа**

Собственные числа и собственные векторы матрицы также являются очень важным ее показателем. Их вычисление может быть полезно, как для общего анализа некоторого линейного оператора, так и в качества важного шага в каком-либо алгоритме. Например, вычисление собственных чисел является выжным шагом при выполнении алгоритма поиска особых точек изображения. Вычисление собственных чисел реализуется с помощью функции `numpy.linalg.eig`.

In [18]:
matrix = np.diag(np.random.randint(1, 5, size=4))
eigen_values, eigen_vectors = np.linalg.eig(matrix)

print(
    f"matrix:\n{matrix}",
    f"eigen values:\n{eigen_values}",
    f"eigen vectors:\n{eigen_vectors}",
    sep="\n\n",
)

matrix:
[[1 0 0 0]
 [0 4 0 0]
 [0 0 1 0]
 [0 0 0 4]]

eigen values:
[1. 4. 1. 4.]

eigen vectors:
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]


## Системы уравнений

Разумеется, одной из важных частей линейной алгебры являются линейные уравнения. Поэтому часть функционала подмодуля `linalg` посвящена решению именно этой проблемы. В этом блоке мы рассмотрим основыные функции, которые могут оказаться полезными при решении практических задач.

**Обращение матрицы**

При решении многих задач возникает необходимость в вычислении обратной матрицы. Такая задача, например, может возникать при вычислении коэффициентов МНК, о чем мы поговорим ниже. В NumPy вычисление обратной матрицы возможно с помощью специальной функции `np.linalg.inv`

In [19]:
matrix = np.random.randint(-10, 10, size=(3, 3))
matrix_inv = np.linalg.inv(matrix)

print(
    f"matrix:\n{matrix}",
    f"matrix inverse:\n{matrix_inv}",
    f"product:\n{np.round(matrix_inv @ matrix, 2)}",
    sep="\n\n",
)

matrix:
[[-7 -6 -4]
 [ 2  4  0]
 [-7  2 -4]]

matrix inverse:
[[ 0.25   0.5   -0.25 ]
 [-0.125  0.     0.125]
 [-0.5   -0.875  0.25 ]]

product:
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]


**Решение системы линейных уравнений**

В части случаев имеется возможность получения точного аналитического решения системы линейных уравнений. В этих случаях система линейных уравнений может быть решена с помощью специальной функции `np.linalg.solve`. Аргументами функции являются матрица коэффициентов и столбец правой части.

Для примера рассмотрим решение системы линейных уравнений:

$$
\begin{equation*}
 \begin{cases}
   x_1 + 2x_2 = 1 \\
   3x_1 + 5x_2 = 2
 \end{cases}
\end{equation*}
$$

In [20]:
equation_matrix = np.array([[1, 2], [3, 5]])
right_part = np.array([1, 2])

solution = np.linalg.solve(equation_matrix, right_part)
print(
    ", ".join([
        f"x{i + 1} = {np.round(solution[i], 2)}"
        for i in range(solution.size)
    ])
)

x1 = -1.0, x2 = 1.0


**МНК**

Стоит отметить, что далеко не всегда существует возможность нахождения точного решения системы линейных уравнений. В реальной задаче может существовать большое количество уравнений, протеворечащих друг другу. В таких условиях ищется оптимальное решение системы, в смысле минимизации некоторой ошибки. Собственно, такая постановка задачи очень напоминает нам постановку задачи МНК. На самом деле МНК можно использовать в качестве метода вычисления оптимального решения системы линейных уравнений, а потому этот алгоритм также реализован в NumPy.

В примере ниже мы рассмотрим функционал NumPy для решения классической задачи, в которой используется МНК - восстановление линейной зависимости.

Предположим, что у нас есть несколько тестовых точек, и мы пытаемся по ним восстановить исходную зависимость вида:
$$y = ax + b, $$

где $a$ и $b$ - искомые коэффициенты.

Пусть известны следующие координаты точек:

In [21]:
abscissa = np.array([0, 1, 2, 3])
ordinates = np.array([-1, 0.2, 0.9, 2.1])

Для использования функции `np.linalg.lstsq` необходимо составить матрицу коэффициентов системы линейных уравнений. Т.е. мы должны свести задачу к решению системы линейных уравнений относительно переменных $a$ и $b$. Обладая указанными выше точками запишем следующую систему:

$$
\begin{equation*}
 \begin{cases}
   ax_1 + b = y_1 \Rightarrow b = -1 \\
   ax_2 + b = y_2 \Rightarrow a + b = 0.2 \\
   ax_3 + b = y_3 \Rightarrow 2a + b = 0.9\\
   ax_4 + b = y_4 \Rightarrow 3a + b = 2.1
 \end{cases}
\end{equation*}
$$

Итак, займемся построением матрицы системы.

In [22]:
equation_matrix = np.vstack((abscissa, np.ones_like(abscissa))).T
print(f"exiation_matrix:\n{equation_matrix}", end="\n\n")

coefficients = np.linalg.lstsq(
    equation_matrix, ordinates, rcond=None
)[0]

print(
    f"incline: {np.round(coefficients[0], 2)}",
    f"shift: {np.round(coefficients[1], 2)}",
    sep="\n",
)

exiation_matrix:
[[0 1]
 [1 1]
 [2 1]
 [3 1]]

incline: 1.0
shift: -0.95


### Задачние 2. МНК

До этого мы рассматривали вычисление коэффициентов МНК с позиции минимизации ошибки. Однако есть еще один, более алгебраический подход к решению проблемы восстановления регрессионной зависимости. В случае МНК для линейной функции мы пытаемся восстановить зависимость вида:

$$y = ax + b$$

Восстановление этой зависимости можно переформулировать, как поиск коэффициентов нашей функции в функциональном базисе $\{1, x\}$, т.е. мы ищем ортогональную проекцию нашей функции $f(x)$ на пространство функций с базисом $\{1, x\}$. 

Обозначим за вектор $\vec{c}$ наш вектор коэффициентов, $A$ - матрица системы, составленная из наблюдений, $\vec{b}$ - вектор-столбец наблюдаемых значений функции. Имеем: 

- $A\vec{c}$ - ортогональная проекция  
- $\vec{b}$ - фактическое значение  
- $A\vec{c} - \vec{b}$ - ортогональная составляющая, т.е. вектор, перпендикулярный всем векторам, к которым было применено преобразование $A$

Отсюда следует, что для любого вектора $\vec{y}$ справедливо:

$$(A\vec{y})^T(A\vec{c} - \vec{b}) = 0$$

Т.е. ортогональная проекция перпендикулярна ортогональной соствляющей, что логично. Поскольку равеноство выполняется для любого $\vec{y}$ раскроем скобки и избавимся от $\vec{y}$:

$$A^TA\vec{c} - A^T\vec{b} = 0$$
$$A^TA\vec{c} = A^T\vec{b}$$
$$\vec{c} = (A^TA)^{-1}A^T\vec{b}$$

Таким образом, мы получили алгоритм вычисления коэффициентов разложения. Т.е. решили задачу восстановления зависимости.

**Часть 1**

Реализуйте описанный алгоритм вычисления коэффициентов регрессии в виде функции. На вход функции подаются массивы абсцисс и ординат. На выходе - массив коэффициентов. Если размеры массивов не совпадают, необходимо возбудить исключение `ShapeMismatchError`.

**Программная реализация**:

In [23]:
class ShapeMismatchError(Exception):
    pass

In [24]:
def get_lsm_coefficients(
    abscissa: np.ndarray,
    ordinates: np.ndarray,
) -> np.ndarray:
    # ваш код

    return np.zeros_like(abscissa)

**Тесты**:

In [25]:
abscissa = np.array([0, 1, 2, 3])
ordinates = np.array([-1, 0.2, 0.9, 2.1])

coefficients = get_lsm_coefficients(abscissa, ordinates)
assert np.allclose(coefficients, np.array([1, -0.95]))

try:
    get_lsm_coefficients(abscissa, np.ones(abscissa.size + 1))

except ShapeMismatchError:
    pass

else:
    assert False, "exception expected"

**Часть 2**

Построение зависимости лишено смысла, если не иметь никакой оценки качества полученных результатов. Одним из показателей качества может выступать коэффициент детерминации, который позволяет опреелить долю дисперсии, которую удалось описать нашей зависимостью. Соответственно, чем выше знчение коэффициента, тем большую долю шума в данных нам удалось описать построенной зависимостью, и тем достоверней результаты. Коэффициент детерминации определяется следующим образом:

$$R^2 = 1 - \frac{\sum^n_{i=1}{(y_i - ax_i - b)^2}}{\sum^n_{i=1}{(y_i - \overline{y})^2}}$$

Реализуйте функцию, которая бы вычисляла коэффициент детерминации, по переданной выборке и полученным коэффициентам.

**Функция**:

In [26]:
def get_determination(
    abscissa: np.ndarray,
    ordinates: np.ndarray,
    coefficients: np.ndarray,
) -> float:
    # ваш код
    return 0

**Тест**:

In [27]:
assert np.isclose(
    get_determination(abscissa, ordinates, coefficients),
    0.99,
    atol=1e-4,
)